In [1]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

In [2]:
# Define class labels
class_labels = {
    1: 'building-no-damage',
    2: 'damaged-building'  # Merging medium, major damage, and total destruction into one class
}

# Define colors for each class for visualization
colors = {
    1: (255, 0, 0),     # Blue for building-no-damage
    2: (255, 0, 255),   # Magenta for damaged-building
}

In [3]:
# Convert mask to bounding boxes
def mask_to_bboxes(mask):
    bboxes = []
    for class_id in np.unique(mask):
        if class_id == 0:  # Skip background
            continue
        # Find contours for the current class
        contours, _ = cv2.findContours((mask == class_id).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            if class_id in [4, 5, 6]:  # Convert damaged classes to a single class id 2
                class_id = 2
            elif class_id == 3:  # Building no damage class
                class_id = 1
            else:
                continue
            bboxes.append((class_id, x, y, x + w, y + h))
    return bboxes

In [4]:
# Convert mask to bounding boxes
def mask_to_bboxes_single(mask_path):
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    bboxes = []
    count = 0
    for class_id in np.unique(mask):
        if class_id == 0:  # Skip background
            continue
        # Find contours for the current class
        contours, _ = cv2.findContours((mask == class_id).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            print(x, y, w, h,class_id)
            if class_id in [4, 5, 6]:  # Convert damaged classes to a single class id 2
                class_id = 2
            elif class_id == 3:  # Building no damage class
                count += 1
                class_id = 1
            else:
                continue
            bboxes.append((class_id, x, y, x + w, y + h))
    print(count)
    return bboxes

In [12]:
# base_dir = 'data'
mask_path = "../data/raw_data/train/train-label-img/10862_lab.png"

print(mask_to_bboxes_single(mask_path))

3392 1952 1 2 1
1158 1707 2842 1293 1
3623 1325 377 348 1
3408 1281 2 1 1
3394 1280 5 1 1
3380 1279 9 1 1
2219 1186 1194 416 1
1082 1060 826 446 1
0 953 751 2026 1
3168 0 832 482 1
0 0 2797 349 1
3524 2001 474 995 3
2986 1927 447 923 1
2591 1891 392 915 1
1788 1856 747 906 1
1303 2042 521 871 4
2337 1612 336 122 7
809 239 602 162 7
186 174 321 138 7
2801 0 152 215 7
0 0 4000 3000 8
1
[(1, 3524, 2001, 3998, 2996), (2, 1303, 2042, 1824, 2913)]


In [15]:
import os

# Define the path to the annotations file
annotations_file = 'bboxes/yolo_annotations_full.txt'

# Create directories if they do not exist
os.makedirs('labels/train', exist_ok=True)
os.makedirs('labels/val', exist_ok=True)
os.makedirs('labels/test', exist_ok=True)

# Function to get the new class label
def get_new_class_label(old_class):
    if old_class == 3:
        return 0
    elif old_class in [4, 5, 6]:
        return 1
    else:
        return None

# Dictionary to keep track of the images processed
processed_images = {}

# Read the annotations file
with open(annotations_file, 'r') as file:
    lines = file.readlines()

# Process each line
for line in lines:
    parts = line.strip().split()
    image_path = parts[0]
    class_label = int(parts[1])
    bbox_info = parts[2:]

    # Get the new class label
    new_class_label = get_new_class_label(class_label)
    if new_class_label is None:
        # Track images that should have empty annotation files
        if image_path not in processed_images:
            processed_images[image_path] = []
        continue

    # Extract the directory and file name
    directory = image_path.split('/')[-3]  # Adjusted to get the correct subdirectory
    file_name = os.path.splitext(os.path.basename(image_path))[0] + '.txt'  # Fixed file name extraction
    output_dir = f'labels/{directory}'
    output_path = os.path.join(output_dir, file_name)

    # Track the bounding box information for the image
    if image_path not in processed_images:
        processed_images[image_path] = []
    processed_images[image_path].append(f"{new_class_label} {' '.join(bbox_info)}\n")

# Write the bounding box information to the corresponding files
for image_path, bboxes in processed_images.items():
    directory = image_path.split('/')[-3]
    file_name = os.path.splitext(os.path.basename(image_path))[0] + '.txt'
    output_dir = f'labels/{directory}'
    output_path = os.path.join(output_dir, file_name)
    
    with open(output_path, 'w') as out_file:
        for bbox in bboxes:
            out_file.write(bbox)

# Ensure every image has a corresponding text file even if empty
all_images = set()
for line in lines:
    image_path = line.strip().split()[0]  # Corrected index to get the image path
    all_images.add(image_path)

for image_path in all_images:
    directory = image_path.split('/')[-3]
    file_name = os.path.splitext(os.path.basename(image_path))[0] + '.txt'
    output_dir = f'labels/{directory}'
    output_path = os.path.join(output_dir, file_name)
    
    # Create an empty file if it does not exist
    if not os.path.exists(output_path):
        open(output_path, 'w').close()

print("Processing complete.")

Processing complete.
